# 📊 Fase 3 – Entrenamiento de Modelo DL (Continuación)

## 🎯 Objetivo
Construir un clasificador utilizando un modelo LSTM RNN optimizado con Nadam para diferenciar entre tráfico **normal** y **sospechoso** en redes monitoreadas, como parte del sistema modular de CiberVigIA, específicamente utilizando el dataset CIC.

## 🧠 Algoritmo sugerido
- **LSTM RNN con Nadam**: Adecuado para datos secuenciales como el tráfico de red, Nadam es un optimizador eficiente.

## ⚙️ Pasos técnicos

1. **Preparación del dataset (CIC)**
   - Cargar el dataset CIC (`/content/sample_data/cic_ids2017_clean.csv`).
   - Preprocesamiento del dataset para adecuarlo a la entrada de la red LSTM (normalización, posible secuenciación).
   - División en conjunto de entrenamiento y prueba (80/20).
   - Separación de variables predictoras (`X`) y etiqueta (`y`).
2. **Construcción y entrenamiento del modelo LSTM RNN**
   - Definir la arquitectura del modelo LSTM (capas, unidades, activación).
   - Compilar el modelo con la función de pérdida adecuada (binary crossentropy) y el optimizador Nadam.
   - Entrenar el modelo con los datos de entrenamiento.

3. **Evaluación del modelo**
   - Evaluar el rendimiento del modelo en el conjunto de prueba.
   - Calcular métricas relevantes (accuracy, precision, recall, F1-score).

4. **Persistencia del modelo**
   - Guardar el modelo entrenado para futuras fases.

## 📦 Entregable

Modelo entrenado con un rendimiento aceptable sobre el conjunto de prueba, documentando la arquitectura, hiperparámetros y métricas obtenidas.

> 💡 *Nota técnica: Para datasets desbalanceados, considera aplicar técnicas como el ajuste de pesos de clase durante el entrenamiento del modelo LSTM.*

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.utils import to_categorical
import gc  # Para liberar memoria

### Paso 1. Cargar el dataset preprocesado

In [2]:
df = pd.read_csv('../../data/processed/cic_ids2017_clean.csv')
print("Forma del dataset preprocesado:", df.shape)
print("Balance de labels:", df['Label'].value_counts())

Forma del dataset preprocesado: (6771538, 79)
Balance de labels: Label
Heartbleed                    2367953
BENIGN                        2367953
SSH-Patator                    296766
DoS GoldenEye                  248503
PortScan                       223943
Infiltration                   199964
Bot                            198047
DoS Hulk                       172849
FTP-Patator                    143176
DDoS                           130288
Web Attack � Sql Injection      98545
Web Attack � Brute Force        97866
DoS slowloris                   90639
DoS Slowhttptest                86492
Web Attack � XSS                48554
Name: count, dtype: int64


### Paso 2: Preparar datos

In [3]:
y = df['Label']
X = df.drop('Label', axis=1)

Encoding para el label

In [4]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_encoded = le.fit_transform(y)
num_classes = len(le.classes_)
y_categorical = to_categorical(y_encoded, num_classes=num_classes)
print("Clases en orden encoded (0 a n):", le.classes_)
label_for_0 = le.inverse_transform([0])[0]
print("Vector 0 corresponde a:", label_for_0)

Clases en orden encoded (0 a n): ['BENIGN' 'Bot' 'DDoS' 'DoS GoldenEye' 'DoS Hulk' 'DoS Slowhttptest'
 'DoS slowloris' 'FTP-Patator' 'Heartbleed' 'Infiltration' 'PortScan'
 'SSH-Patator' 'Web Attack � Brute Force' 'Web Attack � Sql Injection'
 'Web Attack � XSS']
Vector 0 corresponde a: BENIGN


Reshape para LSTM (3D: samples, timesteps=1, features)

In [5]:
X_np = X.values  # A numpy
X_reshaped = X_np.reshape((X_np.shape[0], 1, X_np.shape[1]))

Split train/test/validation

In [6]:
X_train, X_temp, y_train, y_temp = train_test_split(X_reshaped, y_categorical, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
print("Formas: Train", X_train.shape, "Val", X_val.shape, "Test", X_test.shape)

Formas: Train (4740076, 1, 78) Val (1015731, 1, 78) Test (1015731, 1, 78)


In [7]:
gc.collect()

0

### Paso 3: Construir Modelo LSTM RNN

In [13]:
model = Sequential()
model.add(LSTM(units=64, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units=64, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=num_classes, activation='softmax'))

Optimizador Nadam

In [14]:
optimizer = Nadam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

Resumen

In [15]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_4 (LSTM)                   │ (None, 1, 64)          │        36,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 1, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 15)             │           495 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 72,207 (282.06 KB)

 Trainable params: 72,207 (282.06 KB)

 Non-trainable params: 0 (0.00 B)

### Paso 4: Entrenar

In [ ]:
history = model.fit(X_train, y_train, epochs=20, batch_size=128, validation_data=(X_val, y_val), verbose=1)

Epoch 1/20
